In [2]:
#@title Install required libraries
!pip install groq openai -q

import os
import json
from openai import OpenAI
from google.colab import userdata

print("✅ Libraries installed.")

✅ Libraries installed.


In [3]:
#@title Configure Groq API Client
# --- IMPORTANT ---
# Please add your Groq API key to the Colab Secrets Manager.
# Name: GROQ_API_KEY
# Value: your_gq-xxxxxxxx_key

try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    client = OpenAI(
        api_key=GROQ_API_KEY,
        base_url="https://api.groq.com/openai/v1",
    )
    print("✅ Groq client configured successfully.")
except Exception as e:
    print("🚨 Error configuring Groq client.")
    print("Please ensure your GROQ_API_KEY is set correctly in Colab Secrets.")

✅ Groq client configured successfully.


In [19]:
#@title Helper Function: Summarization
def summarize_conversation(history: list[dict]) -> str:
    """Calls the Groq API to summarize the conversation history."""
    if not history:
        return "No conversation to summarize."

    # Format the history for the prompt
    conversation_text = "\n".join(f"{msg['role']}: {msg['content']}" for msg in history)

    prompt = (
        "You are a helpful assistant. Summarize the following conversation into a "
        "concise paragraph. This summary will be used as a memory for a chatbot. "
        "Capture the key topics, user needs, and any resolutions."
    )

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": conversation_text}
            ],
            temperature=0.2,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error during summarization: {e}"



In [20]:
#@title Helper Function: Truncation
def truncate_history(history: list[dict], method: str, limit: int) -> list[dict]:

    if method == 'turns':
        if len(history) <= limit:
            return history
        return history[-limit:] # Keep the last 'limit' messages
    elif method == 'length':
        current_length = 0
        truncated_history = []
        for message in reversed(history):
            current_length += len(message['content'])
            if current_length > limit:
                break
            truncated_history.insert(0, message)
        return truncated_history
    else:
        raise ValueError("Invalid truncation method. Use 'turns' or 'length'.")

In [21]:
#@title The ConversationManager Class
class ConversationManager:
    """
    Manages a conversation, including history and periodic summarization.
    """
    def __init__(self, k_turns_for_summary: int = 3):
        self.history = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]
        self.k = k_turns_for_summary
        self.user_turn_count = 0

    def add_message(self, role: str, content: str):
        """Adds a message and triggers summarization if k-th turn is reached."""
        self.history.append({"role": role, "content": content})

        if role == 'user':
            self.user_turn_count += 1
            if self.user_turn_count > 0 and self.user_turn_count % self.k == 0:
                print(f"\n🤖 --- Reached {self.user_turn_count} user turns. Performing periodic summarization... ---")
                summary = summarize_conversation(self.history)
                # Replace history with a new system prompt and the summary
                self.history = [
                    {"role": "system", "content": "This is a summary of the previous conversation."},
                    {"role": "assistant", "content": summary}
                ]
                print("✅ History has been summarized.")

    def display_history(self):
        """Prints the current conversation history in a readable format."""
        print("\n--- Current Conversation History ---")
        for msg in self.history:
            print(f"[{msg['role'].upper()}]: {msg['content']}")
        print("------------------------------------\n")

In [22]:
#@title Demonstration: Periodic Summarization (k=3)

# Initialize the manager to summarize after every 3rd user message.
convo_manager = ConversationManager(k_turns_for_summary=3)
convo_manager.display_history()

# --- Conversation Turn 1 ---
convo_manager.add_message("user", "Hi, I need help with my internet connection. It's very slow.")
convo_manager.add_message("assistant", "I can help with that. Have you tried restarting your router?")
convo_manager.display_history()

# --- Conversation Turn 2 ---
convo_manager.add_message("user", "Yes, I did. It didn't help. Can you check from your end?")
convo_manager.add_message("assistant", "Of course. I am running diagnostics now. It seems there's an outage in your area.")
convo_manager.display_history()

# --- Conversation Turn 3 (Triggers Summarization) ---
print("🗣️ Adding the 3rd user message, which will trigger summarization...")
convo_manager.add_message("user", "Oh, an outage? Do you have an ETA for when it will be fixed?")
convo_manager.add_message("assistant", "We expect services to be restored in the next 2-3 hours. I'll set up a notification for you.")
convo_manager.display_history()

# --- Conversation Turn 4 (Starts with summarized history) ---
convo_manager.add_message("user", "Okay, please do. Thanks for the help.")
convo_manager.display_history()


--- Current Conversation History ---
[SYSTEM]: You are a helpful assistant.
------------------------------------


--- Current Conversation History ---
[SYSTEM]: You are a helpful assistant.
[USER]: Hi, I need help with my internet connection. It's very slow.
[ASSISTANT]: I can help with that. Have you tried restarting your router?
------------------------------------


--- Current Conversation History ---
[SYSTEM]: You are a helpful assistant.
[USER]: Hi, I need help with my internet connection. It's very slow.
[ASSISTANT]: I can help with that. Have you tried restarting your router?
[USER]: Yes, I did. It didn't help. Can you check from your end?
[ASSISTANT]: Of course. I am running diagnostics now. It seems there's an outage in your area.
------------------------------------

🗣️ Adding the 3rd user message, which will trigger summarization...

🤖 --- Reached 3 user turns. Performing periodic summarization... ---
✅ History has been summarized.

--- Current Conversation History ---
[S

In [23]:
#@title Demonstration: Truncation Options

# Create a longer sample history for demonstration
long_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather like today?"},
    {"role": "assistant", "content": "It's sunny with a high of 25°C."},
    {"role": "user", "content": "Great. Can you also book me a flight to Paris for next week?"},
    {"role": "assistant", "content": "Certainly. Which dates and airline would you prefer?"},
    {"role": "user", "content": "Let's do the 15th to the 22nd on Air France."},
    {"role": "assistant", "content": "Booking is confirmed. Your e-tickets will be sent to your email."}
]

print(f"Original history has {len(long_history)} messages.")

# a) Truncate by number of turns (last 4 messages)
truncated_by_turns = truncate_history(long_history, method='turns', limit=4)
print(f"\n--- Truncated by Turns (last 4) ---")
for msg in truncated_by_turns:
    print(f"[{msg['role'].upper()}]: {msg['content']}")

# b) Truncate by character length (last 200 characters)
truncated_by_length = truncate_history(long_history, method='length', limit=200)
print(f"\n--- Truncated by Length (approx. last 200 chars) ---")
for msg in truncated_by_length:
    print(f"[{msg['role'].upper()}]: {msg['content']}")

Original history has 7 messages.

--- Truncated by Turns (last 4) ---
[USER]: Great. Can you also book me a flight to Paris for next week?
[ASSISTANT]: Certainly. Which dates and airline would you prefer?
[USER]: Let's do the 15th to the 22nd on Air France.
[ASSISTANT]: Booking is confirmed. Your e-tickets will be sent to your email.

--- Truncated by Length (approx. last 200 chars) ---
[ASSISTANT]: Certainly. Which dates and airline would you prefer?
[USER]: Let's do the 15th to the 22nd on Air France.
[ASSISTANT]: Booking is confirmed. Your e-tickets will be sent to your email.


In [12]:
#@title Define the JSON Schema and Tool for Information Extraction

# This schema describes the function and the parameters it accepts.
# The model will generate a JSON object matching these parameters.
user_details_tool = [
    {
        "type": "function",
        "function": {
            "name": "extract_user_info",
            "description": "Extracts user information from a chat message.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The full name of the user."
                    },
                    "email": {
                        "type": "string",
                        "description": "The email address of the user."
                    },
                    "phone": {
                        "type": "string",
                        "description": "The phone number of the user."
                    },
                    "location": {
                        "type": "string",
                        "description": "The city or address of the user."
                    },
                    "age": {
                        "type": "integer",
                        "description": "The age of the user."
                    }
                },
                "required": ["name", "email"]
            }
        }
    }
]

print("✅ JSON schema and tool definition created.")

✅ JSON schema and tool definition created.


In [17]:
#@title Function for JSON Extraction
def extract_user_details(chat_message: str, tools: list) -> dict:

    try:
        response = client.chat.completions.create(
            # Using a more powerful model for better function calling accuracy
            model="llama-3.3-70b-versatile", # Changed to a supported model
            messages=[
                {"role": "system", "content": "You are a data extraction expert."},
                {"role": "user", "content": chat_message}
            ],
            tools=tools,
            tool_choice="auto", # Let the model decide when to call the function
            temperature=0.0
        )

        tool_calls = response.choices[0].message.tool_calls
        if tool_calls:
            # The model decided to call our function
            function_args = json.loads(tool_calls[0].function.arguments)
            return function_args
        else:
            return {"message": "Model did not find information to extract."}

    except Exception as e:
        return {"error": f"An API call failed: {e}"}

In [18]:
#@title Demonstration: Parsing Sample Chats

sample_chats = [
    # Sample 1: All information provided clearly
    "Hi, my name is John Doe. Please sign me up. My email is john.d@example.com and you can reach me at 555-123-4567. I live in New York and I am 32 years old.",

    # Sample 2: Missing some information (phone, age)
    "Hello, I'm Jane Smith and my email is jane.smith@email.net. I'm based in London.",

    # Sample 3: Information is conversational and non-sequential
    "I'm 25 and I need to update my details. My phone is 555-987-6543. Oh, and my name is Alex Ray. My new address is in Tokyo. My email is a.ray@web.com."
]

for i, chat in enumerate(sample_chats):
    print(f"--- Processing Sample Chat #{i+1} ---")
    print(f"User Message: \"{chat}\"")

    extracted_data = extract_user_details(chat, user_details_tool)

    # Pretty-print the validated JSON output
    print("✅ Extracted Information:")
    print(json.dumps(extracted_data, indent=2))
    print("-" * 35 + "\n")

--- Processing Sample Chat #1 ---
User Message: "Hi, my name is John Doe. Please sign me up. My email is john.d@example.com and you can reach me at 555-123-4567. I live in New York and I am 32 years old."
✅ Extracted Information:
{
  "age": 32,
  "email": "john.d@example.com",
  "location": "New York",
  "name": "John Doe",
  "phone": "555-123-4567"
}
-----------------------------------

--- Processing Sample Chat #2 ---
User Message: "Hello, I'm Jane Smith and my email is jane.smith@email.net. I'm based in London."
✅ Extracted Information:
{
  "email": "jane.smith@email.net",
  "location": "London",
  "name": "Jane Smith"
}
-----------------------------------

--- Processing Sample Chat #3 ---
User Message: "I'm 25 and I need to update my details. My phone is 555-987-6543. Oh, and my name is Alex Ray. My new address is in Tokyo. My email is a.ray@web.com."
✅ Extracted Information:
{
  "age": 25,
  "email": "a.ray@web.com",
  "location": "Tokyo",
  "name": "Alex Ray",
  "phone": "555-9